<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/Aug_2_clinical_Biobert_V1_0%20-%20NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
#os.environ[‘CUDA_LAUNCH_BLOCKING’] = 1

In [ ]:
#import pandas as pd
#path='/content/drive/My Drive/NLP/ner - custom.csv'
#data=pd.read_csv(path)
#data.tail(10)

In [3]:
import pandas as pd
import csv
path='/content/drive/My Drive/NLP/SciBert/BC2GM-IOB-Train2.xlsx'
data=pd.read_excel(path,delimiter='\t',names=["Words","tags"],encoding='unicode_escape')

In [4]:
data.shape

(4148, 2)

In [5]:
#print
data.head(50)

,Words,tags
0,Immunohistochemical,O
1,staining,O
2,was,O
3,positive,O
4,for,O
5,S,B-GENE
6,-,I-GENE
7,100,I-GENE
8,in,O
9,all,O


In [6]:
data['tags'].isnull().sum()


28

In [8]:
data['tags']=data['tags'].fillna('O')

In [9]:
data['tags'].isnull().sum()
data['tags'].unique()

array(['O', 'B-GENE', 'I-GENE'], dtype=object)

In [10]:
sentences1=[]
tags1=[]
words=data['Words'].values.tolist()
tag=data['tags'].values.tolist()
#words
temp_sen=[]
temp_tag=[]
count=0
for i in words:
  #print(i)
  
  if i != 'End':
    temp_sen.append(str(i))
    temp_tag.append(str(tag[count]))
    count+=1
  else:
    sentences1.append(temp_sen)
    tags1.append(temp_tag)
    temp_sen=[]
    temp_tag=[]



In [11]:
 type(sentences1[0][2])

str

In [12]:
print(len(tags1))
print(len(sentences1))

111
111


In [13]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [14]:
#map the tags with integers -> tags refer to ner
#data["tag"]:retrieves unique tags, set(data["tag"]) -> refers to set of distinct values
tag_values=list(set(data["tags"].values))
tag_values.append("PAD")
#create a dictionary of tag and values
tag2idx={t:i for i,t in enumerate(tag_values)}

In [15]:
print(tag2idx)
#Tags available :
# I-GENE,B-GENE

{'I-GENE': 0, 'O': 1, 'B-GENE': 2, 'PAD': 3}


In [16]:
!pip3 install transformers
#transformers in library from huggingface

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 890kB 17.5MB/s 
     |████████████████████████████████| 1.1MB 30.1MB/s 
     |████████████████████████████████| 3.0MB 37.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=94cc4b1c0d35f8d6c9becf467af552e15ebc5003f289efa6abe3084edc437026
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [17]:
#Apply the Bert
#use pytorch implementation of bert
import torch
#pip install --no-cache-dir torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#Dataset from a tensor or array or list or dict - TensorDataset
#DataLoader - represents a Python iterable over a dataset, with support for many datasets
#RandomSampler - A Sampler that returns random indices.
#SequentialSampler - A Sampler that returns indices sequentially.
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
#BertTokenizer -- constructs a tokenizer class - based on wordpiece tokenizer
#Bertconfg Config file for all parameters
#from transformers import BertTokenizer,BertConfig
from transformers import AutoTokenizer,AutoModelForTokenClassification,AutoModel
import logging
logging.basicConfig(level=logging.INFO)


In [18]:
#import clinicalbiobert
scibert_tokenizer=AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
scibert_model=AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

INFO:filelock:Lock 139755800480400 acquired on /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpihq14yix


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/config.json in cache at /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c
INFO:filelock:Lock 139755800480400 released on /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/config.json from cache at /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f4

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/vocab.txt in cache at /root/.cache/torch/transformers/e20dcab02f7204e65e966b14f29431339b9a092314e9160a6fa5ce83de398002.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/e20dcab02f7204e65e966b14f29431339b9a092314e9160a6fa5ce83de398002.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:filelock:Lock 139755800480400 released on /root/.cache/torch/transformers/e20dcab02f7204e65e966b14f29431339b9a092314e9160a6fa5ce83de398002.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/vocab.txt from cache at /root/.cache/torch/transformers/e20dcab02f7204e65e966b14f29431339b9a092314e9160a6fa5ce83de398002.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/added_tokens.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/special_tokens_map.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/tokenizer_config.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBER

INFO:transformers.file_utils:storing https://cdn.huggingface.co/emilyalsentzer/Bio_ClinicalBERT/pytorch_model.bin in cache at /root/.cache/torch/transformers/91c95541a292980d2ab0942afab21efcaabbcb767f850bdac4b829126f15b35c.7c0f86182853d46a0efe8b1427d50fe6421bb8346d105db81ee98e35f4b35e14
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/91c95541a292980d2ab0942afab21efcaabbcb767f850bdac4b829126f15b35c.7c0f86182853d46a0efe8b1427d50fe6421bb8346d105db81ee98e35f4b35e14
INFO:filelock:Lock 139753339060968 released on /root/.cache/torch/transformers/91c95541a292980d2ab0942afab21efcaabbcb767f850bdac4b829126f15b35c.7c0f86182853d46a0efe8b1427d50fe6421bb8346d105db81ee98e35f4b35e14.lock
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/emilyalsentzer/Bio_ClinicalBERT/pytorch_model.bin from cache at /root/.cache/torch/transformers/91c95541a292980d2ab0942afab21efcaabbcb767f850bdac4b829126f15b35c.7c0f86182853d46a0efe8b1427d50fe6421bb8

INFO:transformers.modeling_utils:All model checkpoint weights were used when initializing BertModel.

INFO:transformers.modeling_utils:All the weights of BertModel were initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use BertModel for predictions without further training.


In [19]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [20]:
MAX_LEN = 75
Batch_Size = 32

In [ ]:
#device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
#n_gpu=torch.cuda.device_count()

In [ ]:
#split words into sub toekns
#We have to deal with the issue of splitting our token-level labels to related subtokens

In [21]:
def token_and_preserve_labels(sentence,text_labels):
  tokenized_sentence=[]
  labels=[]

  for word,label in zip(sentence,text_labels):
    #tokenize the word and count of # of subwords that word is toeknized into 
    #demonstrators ['demons', '##tra', '##tors']
    tokenized_word=scibert_tokenizer.tokenize(word)
    n_subwords=len(tokenized_word)

    # Add the tokenized word to the final tokenized word list
    tokenized_sentence.extend(tokenized_word)

    # Add the same label to the new list of labels `n_subwords` times
    labels.extend([label] * n_subwords)

  return tokenized_sentence,labels 


In [22]:
#sentence and corresponsing labels 
#sentence form : [list of words]
#labels form : #labels form [ list of ner tags]
tokenized_texts_and_labels=[token_and_preserve_labels(sent,labels) for sent,labels in zip(sentences1,tags1)]

In [23]:
print(tokenized_texts_and_labels)

[(['im', '##mu', '##no', '##his', '##to', '##chemical', 'stain', '##ing', 'was', 'positive', 'for', 's', '-', '100', 'in', 'all', '9', 'cases', 'stained', ',', 'positive', 'for', 'h', '##mb', '-', '45', 'in', '9', '(', '90', '%', ')', 'of', '10', ',', 'and', 'negative', 'for', 'c', '##yt', '##oker', '##ati', '##n', 'in', 'all', '9', 'cases', 'in', 'which', 'my', '##x', '##oid', 'me', '##lan', '##oma', 'remained', 'in', 'the', 'block', 'after', 'previous', 'sections', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['ch', '##lora', '##mp', '##hen', '##ico', '##l', 'ace', '##ty', '##lt', '##ran', '##s', '##fer', '##ase', 'ass', '##ays', 'examining', 'the', 

In [24]:
# split text and labels
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]

labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [25]:
print(tokenized_texts)

[['im', '##mu', '##no', '##his', '##to', '##chemical', 'stain', '##ing', 'was', 'positive', 'for', 's', '-', '100', 'in', 'all', '9', 'cases', 'stained', ',', 'positive', 'for', 'h', '##mb', '-', '45', 'in', '9', '(', '90', '%', ')', 'of', '10', ',', 'and', 'negative', 'for', 'c', '##yt', '##oker', '##ati', '##n', 'in', 'all', '9', 'cases', 'in', 'which', 'my', '##x', '##oid', 'me', '##lan', '##oma', 'remained', 'in', 'the', 'block', 'after', 'previous', 'sections', '.'], ['ch', '##lora', '##mp', '##hen', '##ico', '##l', 'ace', '##ty', '##lt', '##ran', '##s', '##fer', '##ase', 'ass', '##ays', 'examining', 'the', 'ability', 'of', 'i', '##e', '##86', 'to', 're', '##press', 'activity', 'from', 'the', 'h', '##c', '##m', '##v', 'major', 'i', '##e', 'promoter', 'or', 'activate', 'the', 'h', '##c', '##m', '##v', 'early', 'promoter', 'for', 'the', '2', '.', '2', '-', 'k', '##b', 'class', 'of', 'r', '##nas', 'demonstrated', 'the', 'functional', 'integrity', 'of', 'the', 'i', '##e', '##86', 'pro

In [26]:
print(labels)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE'], ['O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'O', 'O',

In [27]:
input_ids = pad_sequences([scibert_tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [28]:
print(len(input_ids))

111


In [29]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [30]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [31]:
print(attention_masks)

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [32]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.1)

In [33]:
#convert all tensor format
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [34]:
tr_tags.shape

torch.Size([99, 75])

In [35]:
tr_masks.shape

torch.Size([99, 75])

In [36]:
tr_inputs.shape

torch.Size([99, 75])

In [37]:
print(tr_tags)

tensor([[1, 2, 0,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 3, 3, 3],
        [1, 1, 1,  ..., 3, 3, 3],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 3, 3, 3],
        [1, 1, 3,  ..., 3, 3, 3]])


In [38]:
input_ids = pad_sequences([scibert_tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [39]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [40]:
print(tags[0])

[1 1 1 1 1 1 1 1 1 1 1 2 0 0 1 1 1 1 1 1 1 1 2 2 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3
 3]


In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [41]:
print(attention_masks)

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [42]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.1)

In [43]:
#convert all tensor format
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [44]:
print(tr_tags)

tensor([[1, 2, 0,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 3, 3, 3],
        [1, 1, 1,  ..., 3, 3, 3],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 3, 3, 3],
        [1, 1, 3,  ..., 3, 3, 3]])


In [45]:
#train_data in the form - input tokens, attention masks for the corresponding tokens and the tags for the tokens
train_data = TensorDataset(tr_inputs,tr_masks,tr_tags)
#returns random indices from the data source
train_sampler = RandomSampler(train_data)
#dataloader combines the dataset and the sampler 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
#samples elements sequentially in the same order
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data,sampler=valid_sampler, batch_size=32)

In [46]:
for input,mask,tag in train_dataloader:
  print(mask.shape)
  print(input.shape)
  print(tag.shape)

torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([3, 75])
torch.Size([3, 75])
torch.Size([3, 75])


Setup the Bert model for finetuning

Setup the Bert model for finetuning


In [47]:
import transformers
#bertfortokenclassification addsa classifier on top of bert model
from transformers import AutoModelForTokenClassification,AdamW,AutoConfig

In [48]:
transformers.__version__

'3.0.2'

In [49]:
print(len(tag2idx))

4


In [ ]:
#config = AutoConfig.from_pretrained("allenai/scibert_scivocab_uncased",num_labels=len(tag2idx))
#model=AutoModelForTokenClassification.from_pretrained("allenai/scibert_scivocab_uncased")
#model=AutoModelForTokenClassification.from_pretrained()
#bert-base-cased : 12 layer bert model cased
#trained bert model with a layer for classification
#output_attentions:  Whether the model returns attentions weights.
#output_hidden_states :  Whether the model returns all hidden-states.

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/allenai/scibert_scivocab_uncased/config.json from cache at /root/.cache/torch/transformers/199e28e62d2210c23d63625bd9eecc20cf72a156b29e2a540d4933af4f50bda1.4b6b9f5d813f7395e7ea533039e02deb1723d8fd9d8ba655391a01a69ad6223d
INFO:transformers.configuration_utils:Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_siz

In [50]:
from transformers import BertForTokenClassification,AdamW,AutoConfig

model1 = BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=len(tag2idx), output_attentions = False,output_hidden_states = False)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/config.json from cache at /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c
INFO:transformers.configuration_utils:Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size

In [ ]:
#set up the optimizer
#ADAMW is the optimizer
#weight_Decay is regularization parameters
#if limited resources, train only the clasffier and not the weights with the labels


In [ ]:
import torch
#specify GPU as device
if torch.cuda.is_available():
  device=torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla T4 with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Tesla T4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [ ]:
# Tell pytorch to run this model on the GPU.
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [51]:
model1.num_labels

4

In [52]:
model1.parameters

<bound method Module.parameters of BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye

In [53]:
model1.cpu

<bound method Module.cpu of BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [54]:
#Print the different parameters
params=list(model1.named_parameters())

#print
print("the model has {:} number of parameters".format(len(params)))

the model has 201 number of parameters


In [56]:
for p in params[0:5]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---First Transformer---")
for p in params[5:21]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
for p in params[190:200]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---output layer---")
for p in params[-4:]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)
---First Transformer---
bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (768,)
bert.encoder.layer.0.attention.output.LayerNorm.weight        (768,)
bert.encod

In [ ]:
#grab the training hyperparameters from within the stored model.

In [58]:
#grab the optimzer
optimizer=AdamW(model1.parameters(),
               lr=2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5 
               eps=1e-8) #a very small number to prevent any division by zero in the implementation)

For the purposes of fine-tuning, the authors recommend choosing from the following values (from Appendix A.3 of the BERT paper):

Batch size: 16, 32
Learning rate (Adam): 5e-5, 3e-5, 2e-5
Number of epochs: 2, 3, 4

We also add a scheduler to linearly reduce the learning rate throughout the epochs.

> Indented block



In [62]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.


In [63]:
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).

In [64]:
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.

In [66]:
#Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,#deafault value
    num_training_steps=total_steps)




Training Loop : Training/validation for each passes



In [67]:
from sklearn.metrics import f1_score, accuracy_score


Helper function for formatting elapsed times as hh:mm:ss

Kick off the Training

In [68]:
import random
import numpy as np

In [69]:

# Set the seed value all over the place to make this reproducible.
seed_val=42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [70]:
#run in cpu
device = torch.device("cpu")
model1.cpu

<bound method Module.cpu of BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [72]:
#for ech epoch
#Run the training for the given epochs - typically between 2 and 4
#range 0 to 3

loss_values, validation_loss_values = [], []


for _ in trange(epochs,desc="EPOCH"):
  print("")
  #print ---Epoch 1/2 ----
  #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

  #Reset the total loss for this epoch
  total_train_loss=0
  
  #train the model
  model1.train()

  for step,batch in enumerate(train_dataloader):
    print(step)
    #print(batch)

    #add the batch to the GPU
    #batch=tuple(t.to(device) for t in batch)
    #print(batch.shape)
    b_input_ids,b_input_mask,b_labels=batch
    #print(b_labels)
  #always clear the gradients before doing a backward pass
    model1.zero_grad()

   #perform a forward pass
    loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
    
    #logits = logits.detach().cpu().numpy()
    #print("step number -", step)
    #print("score--",logits.shape)
    #outout shape in the form : 32*75*18 --32 batch size,75 --> sequence size, 18 --> Tags 
    #print("argmax--")
    #print("logits--",logits)
    #print("argmax--",np.argmax(logits,axis=1))
   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_train_loss+=loss.item()

   # Perform a backward pass to calculate the gradients.
   #compute gradient of loss w.r.t all the parameters in loss that have requires_grad = True and store them in parameter.grad attribute for every parameter.
    loss.backward()

   # Clip the norm of the gradients to 1.0.
   # This is to help prevent the "exploding gradients" problem
   #within a specific range (clip)
    torch.nn.utils.clip_grad_norm_(model1.parameters(), 1.0)

     
   # Update parameters and take a step using the computed gradient.
   # The optimizer dictates the "update rule"--how the parameters are
   # modified based on their gradients, the learning rate, etc.
   #updates all the parameters based on parameter.grad
    optimizer.step()

   # Update the learning rate.
    scheduler.step()
#Return to the epoch
   # Calculate the average loss over all of the batches.  
  avg_train_loss= total_train_loss/len(train_dataloader)
  print("Average train loss: {}".format(avg_train_loss))
  #append the loss for trainig
  loss_values.append(avg_train_loss)
  print(" Training is completed")
  print("Running Validation... ")

  #put the model in evaluation mode
  model1.eval()

  # Tracking variables 
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0
  predictions , true_labels = [], []

  for batch in valid_dataloader:

    #add the batch to the GPU
    #batch=tuple(t.to(device) for t in batch)
    b_input_ids,b_input_mask,b_labels=batch
   
    #print("validation labels:",b_labels)
     #always clear the gradients
     #in evaluation mode, you can't do backprog, to save time, dont compute the gradients
    with torch.no_grad():
      #perform a forward pass
      loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

    #logits=logits.detach().cpu().numpy()
    #label_ids=b_labels.to('cpu').numpy()
    label_ids=b_labels


   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_eval_loss+=loss.mean().item()
    predictions.extend([list(p) for p in np.argmax(logits,axis=2)])
    true_labels.extend(label_ids)

  eval_loss=total_eval_loss/len(valid_dataloader) 
  validation_loss_values.append(eval_loss)
  print("Validation loss: {}".format(eval_loss))

  pred_tags=[tag_values[p_i] for p,l in zip(predictions,true_labels) for p_i,l_i in zip(p,l) if tag_values[l_i] != "PAD"]

  valid_tags=[tag_values[l_i] for l in true_labels for l_i in l if tag_values[l_i]!="PAD"]

  print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags,average='weighted')))


EPOCH:   0%|          | 0/4 [00:00<?, ?it/s]


Training...
0
1
2
3
Average train loss: 0.9765943884849548
 Training is completed
Running Validation... 



EPOCH:  25%|██▌       | 1/4 [01:17<03:52, 77.38s/it]

Validation loss: 0.5174099802970886
Validation Accuracy: 0.896551724137931
Validation F1-Score: 0.9454545454545454

Training...
0
1
2
3
Average train loss: 0.4018089100718498
 Training is completed
Running Validation... 



EPOCH:  50%|█████     | 2/4 [02:33<02:33, 76.92s/it]

Validation loss: 0.47732412815093994
Validation Accuracy: 0.896551724137931
Validation F1-Score: 0.9454545454545454

Training...
0
1
2
3
Average train loss: 0.38450832664966583
 Training is completed
Running Validation... 



EPOCH:  75%|███████▌  | 3/4 [03:48<01:16, 76.55s/it]

Validation loss: 0.45294085144996643
Validation Accuracy: 0.896551724137931
Validation F1-Score: 0.9454545454545454

Training...
0
1
2
3
Average train loss: 0.4280616268515587
 Training is completed
Running Validation... 



EPOCH: 100%|██████████| 4/4 [05:03<00:00, 75.94s/it]

Validation loss: 0.43615061044692993
Validation Accuracy: 0.896551724137931
Validation F1-Score: 0.9454545454545454


In [ ]:
test_sentence = """
Mr. Trump’s tweets began just moments after a Fox News report by Mike Tobin, a 
reporter for the network, about protests in Minnesota and elsewhere. 
"""

In [ ]:
#tokenize  the text
tokenized_sentence=scibert_tokenizer.encode(test_sentence)
#the output is sequence of ids (token to ids)

In [ ]:
#conver to tensor format
input_ids = torch.tensor([tokenized_sentence])

In [ ]:
#run the sentence through the model
with torch.no_grad():
  output=model1(input_ids)

In [ ]:
label_indices=np.argmax(output[0],axis=2)

In [ ]:
print(label_indices)

tensor([[3, 2, 2, 0, 2, 2, 2, 3, 3, 3, 3, 3, 3, 0, 3, 2, 2, 1, 2, 3, 1, 2, 3, 0,
         2, 2, 3, 2, 0, 0, 2, 0, 3, 2, 1, 0, 3]])


In [ ]:
#join the split tokens
tokens=scibert_tokenizer.convert_ids_to_tokens(input_ids[0])


In [ ]:
print(tokens)

['[CLS]', 'mr', '.', 'tr', '##ump', '’', 's', 'tweets', 'began', 'just', 'moments', 'after', 'a', 'fox', 'news', 'report', 'by', 'mik', '##e', 'to', '##bin', ',', 'a', 'reporter', 'for', 'the', 'network', ',', 'about', 'prote', '##sts', 'in', 'minnesota', 'and', 'elsewhere', '.', '[SEP]']


In [ ]:
new_token,new_labels=[],[]
for token,label_idx in zip(tokens,label_indices[0]):
  if token.startswith("##"):
    #add at the end
    new_token[-1]=new_token[-1]+token[2:]
  else:
    #if its not special token
    new_labels.append(tag_values[label_idx])
    new_token.append(token) 

In [ ]:
for token,label in zip(new_token,new_labels):
  print("{}\t{}".format(token,label))
  

[CLS]	PAD
mr	B-GENE
.	B-GENE
trump	O
’	B-GENE
s	B-GENE
tweets	PAD
began	PAD
just	PAD
moments	PAD
after	PAD
a	PAD
fox	O
news	PAD
report	B-GENE
by	B-GENE
mike	I-GENE
tobin	PAD
,	B-GENE
a	PAD
reporter	O
for	B-GENE
the	B-GENE
network	PAD
,	B-GENE
about	O
protests	O
in	O
minnesota	PAD
and	B-GENE
elsewhere	I-GENE
.	O
[SEP]	PAD


# For each batch of training data...